In [1]:
from typing import Sequence, Tuple, List
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import cirq
from ansatz import CylicQAOAAnsatz
from maxcut import edge_operator, maxcut_hamiltonian, bitstring_energy

In [2]:
def optimize_ansatz(ansatz, gamma, beta):
    def objective_callback(vars: np.ndarray):
        assert vars.size % 2 == 0
        
        gammas = vars[:(vars.size // 2)]
        betas = vars[(vars.size // 2):]
        return ansatz.energy(gammas, betas)

    vars0 = np.concatenate((gamma, beta))
    opt_result = minimize(objective_callback, vars0, method="Powell", options={"maxiter": 1_000_000})
    assert opt_result.success, f"Optimizer failed: {opt_result.message}"
    optimized_energy =  objective_callback(opt_result.x)
    gamma_opt = opt_result.x[:gamma.size]
    beta_opt = opt_result.x[gamma.size:]
    return optimized_energy, gamma_opt, beta_opt


def optmize_ansatz_random_start(ansatz, layers, repetitions):
    gammas = np.random.rand(repetitions, layers)
    betas = np.random.rand(repetitions, layers)
    all_outputs = []
    for i in range(repetitions):
        all_outputs.append(optimize_ansatz(ansatz, gammas[i, :], betas[i, :]))
    energies = [out[0] for out in all_outputs]
    i_opt = np.argmin(energies)
    return all_outputs[i_opt]

In [3]:
n = 10
p = n-5
qs = cirq.LineQubit.range(n)
qubit_graph = nx.Graph()
for i in range(len(qs)):
    qubit_graph.add_edge(qs[i], qs[(i+1) % len(qs)])
hamiltonian = maxcut_hamiltonian(qubit_graph)
ansatz = CylicQAOAAnsatz(qubit_graph, hamiltonian)

no_reference_energy, gamma, beta = optmize_ansatz_random_start(ansatz, p, 10)
print(no_reference_energy)

-9.97135391831398


In [ ]:
rounds = 3
reference = [True] * len(qubit_graph.nodes)
alpha = np.linspace(2.0, 0.0, num=p)
for i in range(rounds):
    print("i = ", i)
    print("reference = ", reference)
    ansatz = CylicQAOAAnsatz(qubit_graph, hamiltonian, reference=reference, alpha=alpha)
    energy, gamma, beta = optmize_ansatz_random_start(ansatz, p, 10)
    print("Energy = ", energy)
    sampled_bitstrings = ansatz.sample_bitstrings(gamma, beta, 1000)
    # TODO keep best bitstrings from last run to use as backup
    print(sampled_bitstrings.shape)
    energies = [bitstring_energy(sampled_bitstrings[i, :], hamiltonian) for i in range(sampled_bitstrings.shape[0])]
    i_best = np.argmin(energies)
    print("Best energy = ", energies[i_best])
    reference = sampled_bitstrings[i_best, :]

i =  0
reference =  [True, True, True, True, True, True, True, True, True, True]
Energy =  -8.849030762910843
(1000, 10)
Best energy =  -10.0
i =  1
reference =  [0 1 0 1 0 1 0 1 0 1]
Energy =  -9.865801185369492
(1000, 10)
Best energy =  -6.0
i =  2
reference =  [0 0 0 1 0 0 1 1 0 1]
Energy =  -8.752063691616058
(1000, 10)
Best energy =  -8.0
